In [404]:
import numpy as np 
import pandas as pd

Có nhiều điều mình không thể hiểu hết về cách đạo hàm của Gradient Descent. <br>
Có những bài đạo hàm phía trái, hoặc đạo hàm phía phải cho kết quả. <br>
Hoặc là sử dụng Numerical Descent lại cho ra cực trị tối ưu - mặc dù không sử dụng momentum...(Mình có code bên dưới) <br>
Bàn luận toán với mình về Gradient Descent <br>

There's many things i can't understand all about how Gradient Descent work. <br>
Some gradient by left, some gradient by right, or using Numerical Descent WITHOUT momentum give us optimal value... (I had code below) <br>
You can help me to improve math in Gradient Descent!

## GRADIENT DESCENT

GRADIENT

In [405]:
def cost(w: np.ndarray) -> np.ndarray:
    """
    Hàm mục tiêu để tìm minimum point
    """
    return w ** 2 + 10 * np.sin(w)

def grad(w: np.ndarray, cost) -> np.ndarray:
    """
    Đạo hàm bên phải hàm cost 
    """
    w_grad = np.zeros_like(w)
    epsilon = 1e-3
    for i in range(w.shape[0]):
        for j in range(w.shape[1]):
            w_right = w.copy()
            w_right[i, j] += epsilon
            w_grad[i, j] = (cost(w_right) - cost(w)) / epsilon
    return w_grad

def grad_left(w: np.ndarray, cost) -> np.ndarray:
    """
    Đạo hàm bên trái hàm cost 
    """
    w_grad = np.zeros_like(w)
    epsilon = 1e-3
    for i in range(w.shape[0]):
        for j in range(w.shape[1]):
            w_left = w.copy()
            w_left[i, j] -= epsilon
            w_grad[i, j] = (cost(w) - cost(w_left)) / epsilon
    return w_grad

def numerical_grad(w: np.ndarray, cost) -> np.ndarray:
    """
    w(right) - w(left) / 2 * epsilon
    Đạo hàm này là đạo hàm từng phần - đạo hàm 2 phía
    """
    w_grad = np.zeros_like(w)
    epsilon = 1e-3
    for i in range(w.shape[0]):
        for j in range(w.shape[1]):
            w_right = w.copy()
            w_left = w.copy()
            w_right[i, j] += epsilon
            w_left[i, j] -= epsilon
            w_grad[i, j] = (cost(w_right) - cost(w_left)) / (2 * epsilon)
    return w_grad

def check_converged(theta, gradFunc, cost, epsilon=1e-3):
    """
    Kiểm tra sự hội tụ của đạo hàm tại điểm theta
    """
    return np.linalg.norm(gradFunc(theta, cost)) / len(theta) < epsilon

BASIC CONCEPT

In [406]:
def GD(w_init, gradFunc, cost, eta):
    """
        w_init: điểm w khởi tạo, w.init là mảng 2 chiều
        grad: hàm đạo hàm của hàm mục tiêu
        eta: learning rate
    """
    w = [w_init]
    for ep in range(1000):
        w_next = w[-1] - eta * gradFunc(w[-1], cost)
        if check_converged(w_next, gradFunc, cost):
            break
        w.append(w_next)
    
    return w, ep

WITH MOMENTUM

$$
v_t = \gamma v_{t-1} + \eta \bigtriangledown_\theta J (\theta)
$$
$$
\theta = \theta - v_t
$$

In [407]:
def GD_momentum(theta_init, gradFunc, cost, eta, gamma=0.9):
    """
    theta_init: like w_init
    eta: learning rate
    gamma: hệ số - thường là 0.9 nhân với v trước
    v_t = gamma * v_t-1 + eta * grad(theta)
    """
    epochs = 5000
    thetas = [theta_init]
    v = np.zeros_like(theta_init)
    for ep in range(epochs):
        v_next = gamma * v + eta * gradFunc(thetas[-1], cost)
        theta_next = thetas[-1] - v_next
        if check_converged(theta_next, gradFunc, cost):
            break
        v = v_next
        thetas.append(theta_next)
        
    return thetas, ep


implement

In [408]:
theta_init = np.array([[5]]) # w phải có kích thước là (feature, 1)
thetas, ep = GD(theta_init, grad, cost, .1)
print('GD - gradient right: w = ', thetas[-1].T, '\n%d iterations' %(ep+1))
thetas, ep = GD(theta_init, grad_left, cost, .1)
print('GD - gradient left: w = ', thetas[-1].T, '\n%d iterations' %(ep+1))
thetas, ep = GD(theta_init, numerical_grad, cost, .1)
print('GD - numerical gradient: w = ', thetas[-1].T, '\n%d iterations' %(ep+1))
thetas, ep = GD_momentum(theta_init, numerical_grad, cost, .1)
print('GD with momentum - numerical gradient: w = ', thetas[-1].T, '\n%d iterations' %(ep+1))

GD - gradient right: w =  [[3.83628779]] 
6 iterations
GD - gradient left: w =  [[-1.30585389]] 
34 iterations
GD - numerical gradient: w =  [[-1.30635179]] 
31 iterations
GD with momentum - numerical gradient: w =  [[-1.30431627]] 
243 iterations


NESTEROV ACCELERATED GRADIENT (NAG)

$$
v_t = \gamma v_{t-1} + \eta \nabla_\theta J (\theta - \gamma v_{t-1})
$$

$$
\theta = \theta - v_t
$$

In [409]:
def GD_nag(theta_init, gradFunc, cost, eta, gamma=0.9):
    epochs = 100
    thetas = [theta_init]
    v = np.zeros_like(theta_init)
    for ep in range(epochs):
        v_next = gamma * v + eta * gradFunc(thetas[-1] - gamma * v, cost)
        theta = thetas[-1] - v_next
        if check_converged(thetas[-1], gradFunc, cost):
            break
        thetas.append(theta)
        v = v_next
    return thetas, ep

In [410]:
theta_init = np.array([[5]]) # w phải có kích thước là (feature, 1)
thetas, ep = GD_nag(theta_init, grad, cost, .1)
print('GD - gradient right: w = ', thetas[-1].T, '\n%d iterations' %(ep+1))
thetas, ep = GD_nag(theta_init, grad_left, cost, .1)
print('GD - gradient left: w = ', thetas[-1].T, '\n%d iterations' %(ep+1))
thetas, ep = GD_nag(theta_init, numerical_grad, cost, .1)
print('GD - numerical gradient: w = ', thetas[-1].T, '\n%d iterations' %(ep+1))

GD - gradient right: w =  [[5]] 
1 iterations
GD - gradient left: w =  [[-1.30599638]] 
21 iterations
GD - numerical gradient: w =  [[-1.30649585]] 
21 iterations


STOCHASTIC GRADIENT DESCENT

Ưu: Thuật toán hội tụ rất nhanh!

Ý tưởng chính của SGD là chọn điểm dữ liệu để đạo hàm. <br>
Ở bài này, ta cho random ra hoán vị của N datapoint. Chẳng hạn như có 5 dữ liệu là [0, 1, 2, 3, 4], ta hoán vị random thì sẽ ra [3, 1, 2, 4, 0].<br>
Từ đó ta chọn được điểm dữ liệu thứ i để đạo hàm (đọc hàm grad_single() để hiểu rõ hơn.)

In [411]:
X = np.random.rand(1000, 1) # random range(0, 1) with shape (1000, 1)
y = 4 + 3 * X + .2 * np.random.randn(1000, 1)

# f(x) = x * w^T (x.shape = (N, feature), w^T.shape = (feature, 1))
ones = np.ones_like(X)
Xbar = np.concatenate((ones, X), axis=1)
N = len(Xbar)

In [432]:

def lr_cost(w: np.ndarray, X=Xbar, Y=y) -> np.ndarray:
    return .5 / X.shape[0] * np.linalg.norm(Y - X.dot(w), 2) ** 2
    

def grad_single(w: np.ndarray, i: int, permute: np.ndarray) -> np.ndarray:
    """
    Đạo hàm một điểm dữ liệu tại vị trí thứ true_i
    Giả sử permute, i truyền vào lần lượt là [1, 3, 2, 0] và 1
    => true_i = permute[1] = 3
    """
    true_i = permute[i]
    xi = Xbar[true_i, :]
    yi = y[true_i]
    epsilon = 1e-3
    w_grad = np.zeros_like(w)
    for i in range(w.shape[0]):
        for j in range(w.shape[1]):
            w_right = w.copy()
            w_left = w.copy()
            w_right[i, j] += epsilon
            w_left[i, j] -= epsilon
            w_grad[i, j] = (lr_cost(w_right, xi, yi) - lr_cost(w_left, xi, yi)) / (2 * epsilon)
    return w_grad

def SGD(w_init, eta):
    """
    Đi qua giới hạn tối đa 10 epochs
    Khi đó, mỗi epochs sẽ đi qua 1 lần lặp qua N điểm dữ liệu được shuffle mới
    Cứ đi qua 10 điểm dữ liệu, thì w mới được cập nhật và kiểm tra hội tụ
    Nếu chưa hội tụ thì w_toCheck là biến của 10 lần cập nhật w_next trước đó để kiểm tra
    """
    w = [w_init]
    epoch_check = 10
    N = X.shape[0]
    count = 0
    temp_w = w_init
    for ep in range(10):
        permute = np.random.permutation(N)
        for i in range(N):
            count += 1
            gradient = grad_single(w[-1], i, permute)
            w_next = w[-1] - eta * gradient 
            w.append(w_next)
            if count % epoch_check == 0:
                if np.linalg.norm(w_next - temp_w, 2) / len(w_init) < 1e-3:                                    
                    return w, ep
                else: temp_w = w_next
    return w, ep

theta_init = np.array([[5.], [2.]])
thetas, ep = SGD(theta_init, .1)
print('GD - gradient: w = ', thetas[-1].T, '\n%d iterations' %(ep+1))
thetas, ep = GD_nag(theta_init, numerical_grad, lr_cost, .1)
print('GD NAG - gradient: w = ', thetas[-1].T, '\n%d iterations' %(ep+1))

GD - gradient: w =  [[3.99322352 2.92643375]] 
2 iterations
GD NAG - gradient: w =  [[3.98374898 3.01617674]] 
72 iterations
